In [1]:
import tldextract

## 导入数据

In [2]:
X = []
y = []

In [3]:
def get_data(filename, isdga):
    fd = open(filename, encoding='UTF-8-sig')
    for line in fd:
        X.append(tldextract.extract(line).domain) 
        if isdga:
            y.append(1)
        else:
            y.append(0)

In [4]:
get_data('/Users/dqy/My/DGA_detection/alexa.csv',0)
get_data('/Users/dqy/My/DGA_detection/dgaURL.csv',1)

In [5]:
print(X[0:5])
print(y[0:5])
print(X[10000:10005])
print(y[10000:10005])
print(len(X))
print(len(y))

['google', 'facebook', 'youtube', 'yahoo', 'baidu']
[0, 0, 0, 0, 0]
['gllcsbqrbfch', 'hkgjjokkwodt', 'jjnehudfmclq', 'hieffvbcjitc', 'fckjmggtxchv']
[1, 1, 1, 1, 1]
20000
20000


## 向量化

In [6]:
from keras.layers.core import Dense
from keras.models import Sequential
import sklearn
from sklearn import feature_extraction
from sklearn.model_selection import train_test_split 

Using TensorFlow backend.


In [7]:
# Create feature vectors
print ("vectorizing data")
ngram_vectorizer = feature_extraction.text.CountVectorizer(analyzer='char', ngram_range=(2, 2))
count_vec = ngram_vectorizer.fit_transform(X)

vectorizing data


## 训练模型

In [8]:
acc = []
rec = []
X_train, X_hold, y_train, y_hold, = train_test_split(count_vec, y, test_size=0.05)
nfolds=10
batch_size=128
max_features = count_vec.shape[1]

In [9]:
#k_folds交叉验证
for fold in range(nfolds):
    print ("fold %u/%u" % (fold+1, nfolds))
    X_train, X_test, y_train, y_test, = train_test_split(X_train, y_train, test_size=1/nfolds)
    
    #建立logistic regression模型
    print ('Build model...')
    model = Sequential()
    model.add(Dense(1, input_dim=max_features, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam')
    
    print ("Train...")
    model.fit(X_train.todense(), y_train, batch_size=batch_size, epochs=10)
    probs = model.predict_proba(X_test.todense())
    acc.append(sklearn.metrics.accuracy_score(y_test, probs > .5))
    rec.append(sklearn.metrics.recall_score(y_test, probs > .5))

fold 1/10
Build model...
Train...
Epoch 1/10
17100/17100 [==============================] - 1s 43us/step - loss: 0.5982
Epoch 2/10
17100/17100 [==============================] - 0s 26us/step - loss: 0.4381
Epoch 3/10
17100/17100 [==============================] - 0s 27us/step - loss: 0.3420: 0s - loss: 0.343
Epoch 4/10
17100/17100 [==============================] - 0s 22us/step - loss: 0.2800
Epoch 5/10
17100/17100 [==============================] - 0s 23us/step - loss: 0.2372
Epoch 6/10
17100/17100 [==============================] - 0s 16us/step - loss: 0.2060
Epoch 7/10
17100/17100 [==============================] - 0s 17us/step - loss: 0.1822
Epoch 8/10
17100/17100 [==============================] - 0s 21us/step - loss: 0.1634
Epoch 9/10
17100/17100 [==============================] - 0s 18us/step - loss: 0.1482
Epoch 10/10
17100/17100 [==============================] - 0s 19us/step - loss: 0.1356
fold 2/10
Build model...
Train...
Epoch 1/10
15390/15390 [=============================

6623/6623 [==============================] - 0s 17us/step - loss: 0.5603
Epoch 3/10
6623/6623 [==============================] - 0s 17us/step - loss: 0.4943
Epoch 4/10
6623/6623 [==============================] - 0s 17us/step - loss: 0.4407
Epoch 5/10
6623/6623 [==============================] - 0s 24us/step - loss: 0.3966
Epoch 6/10
6623/6623 [==============================] - 0s 17us/step - loss: 0.3602
Epoch 7/10
6623/6623 [==============================] - 0s 20us/step - loss: 0.3296
Epoch 8/10
6623/6623 [==============================] - 0s 17us/step - loss: 0.3037
Epoch 9/10
6623/6623 [==============================] - 0s 19us/step - loss: 0.2815
Epoch 10/10
6623/6623 [==============================] - 0s 17us/step - loss: 0.2623


## 打印准确率和召回率

In [10]:
print("fold    accuracy      recall")
for fold in range(nfolds):
    print("%4d    %f    %f"%(fold+1, acc[fold], rec[fold]))
probs = model.predict_proba(X_hold.todense())
print("test    %f    %f"%(sklearn.metrics.accuracy_score(y_hold, probs > .5),sklearn.metrics.recall_score(y_hold, probs > .5)))

fold    accuracy      recall
   1    0.987368    0.992375
   2    0.981287    0.985092
   3    0.981157    0.991903
   4    0.981241    0.990113
   5    0.986367    0.993569
   6    0.973262    0.987273
   7    0.974257    0.980469
   8    0.972497    0.993435
   9    0.974328    0.990610
  10    0.966033    0.978320
test    0.981000    0.988417


## test

In [11]:
my_input="gllcsbqrbfch.com"
X.append(tldextract.extract(my_input).domain)
x = ngram_vectorizer.fit_transform(X)[len(X)-1]
my_probs = model.predict(x.todense())
if(my_probs>.5):
    print("%s.com is simda"%X[len(X)-1])
else:
    print("%s.com is benign"%X[len(X)-1])
del X[len(X)-1]

gllcsbqrbfch.com is simda


In [12]:
my_input="windowsupdate.com"
X.append(tldextract.extract(my_input).domain)
x = ngram_vectorizer.fit_transform(X)[len(X)-1]
my_probs = model.predict(x.todense())
if(my_probs>.5):
    print("%s.com is simda"%X[len(X)-1])
else:
    print("%s.com is benign"%X[len(X)-1])
del X[len(X)-1]

windowsupdate.com is benign
